## Text analysis 
### Sentiment Analysis

In [7]:
import pandas as pd
from nltk.sentiment.vader import SentimentIntensityAnalyzer
from nltk import download
import matplotlib.pyplot as plt

df = pd.read_csv("../datasets/raw_analyst_ratings.csv")

sia = SentimentIntensityAnalyzer()

# Apply sentiment analysis
df['Sentiment Scores'] = df['headline'].apply(sia.polarity_scores)
df['Sentiment'] = df['Sentiment Scores'].apply(lambda score: 'positive' if score['compound'] > 0.05 
                                             else ('negative' if score['compound'] < -0.05 
                                             else 'neutral'))

sentiment_distribution = df['Sentiment'].value_counts()
print("Sentiment Distribution:")
print(sentiment_distribution)

# Plot sentiment distribution
sentiment_distribution.plot(kind='bar', title='Sentiment Distribution of Headlines')
plt.xlabel('Sentiment')
plt.ylabel('Number of Headlines')
plt.show()

LookupError: 
**********************************************************************
  Resource [93mvader_lexicon[0m not found.
  Please use the NLTK Downloader to obtain the resource:

  [31m>>> import nltk
  >>> nltk.download('vader_lexicon')
  [0m
  For more information see: https://www.nltk.org/data.html

  Attempted to load [93msentiment/vader_lexicon.zip/vader_lexicon/vader_lexicon.txt[0m

  Searched in:
    - 'C:\\Users\\SQL/nltk_data'
    - 'E:\\Anaconda3\\nltk_data'
    - 'E:\\Anaconda3\\share\\nltk_data'
    - 'E:\\Anaconda3\\lib\\nltk_data'
    - 'C:\\Users\\SQL\\AppData\\Roaming\\nltk_data'
    - 'C:\\nltk_data'
    - 'D:\\nltk_data'
    - 'E:\\nltk_data'
    - ''
**********************************************************************


### Topic Modeling 

In [9]:
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.decomposition import LatentDirichletAllocation
import spacy

# Load dataset
df = pd.read_csv("../datasets/raw_analyst_ratings.csv")

# Preprocess text: Tokenize and remove stopwords
nlp = spacy.load("en_core_web_sm")


def preprocess_text(text):
    doc = nlp(text)
    tokens = [token.lemma_ for token in doc if not token.is_stop and not token.is_punct]
    return " ".join(tokens)


df["clean_headline"] = df["headline"].apply(preprocess_text)

# Vectorize the headlines
vectorizer = CountVectorizer(max_df=0.95, min_df=2, stop_words="english")
X = vectorizer.fit_transform(df["clean_headline"])

# Perform LDA topic modeling
lda = LatentDirichletAllocation(n_components=5, random_state=42)
lda.fit(X)


# Display the topics
def display_topics(model, feature_names, no_top_words):
    for topic_idx, topic in enumerate(model.components_):
        print(f"Topic #{topic_idx}:")
        print(
            " ".join(
                [feature_names[i] for i in topic.argsort()[: -no_top_words - 1 : -1]]
            )
        )
        print()


no_top_words = 10
display_topics(lda, vectorizer.get_feature_names_out(), no_top_words)

ModuleNotFoundError: No module named 'spacy'